## Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

### Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

llm = ChatOpenAI(model="gpt-4o-mini")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [3]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_40524/3660245525.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools,


Lets have a look at the ReAct template

In [4]:
agent.invoke(input="How many members does the A Team have?")



> Entering new AgentExecutor chain...
To answer this question, I need to recall what I know about the fictional group known as the A-Team. The A-Team typically consists of four main members: Hannibal Smith, Face (Templeton Peck), B.A. Baracus, and Murdock (H.M. Murdock). Therefore, I can conclude the number of members in the A-Team.

Final Answer: 4 members.

> Finished chain.


{'input': 'How many members does the A Team have?', 'output': '4 members.'}

In [5]:
agent.invoke(input="What is 100 devided by 25?")



> Entering new AgentExecutor chain...
To find the result of 100 divided by 25, I will perform the division operation.  
Action: Calculator  
Action Input: 100 / 25  
Observation: Answer: 4.0
Thought:I now know the final answer  
Final Answer: 4.0

> Finished chain.


{'input': 'What is 100 devided by 25?', 'output': '4.0'}

### Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

In [7]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name: str = "restaurant search"
    description: str = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.invoke(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [8]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

In [10]:
agent.invoke(input="Who are the members of the A-Team?")



> Entering new AgentExecutor chain...
The A-Team is a fictional group of characters from the television series of the same name. The main members include:
1. Colonel John "Hannibal" Smith
2. Lieutenant Templeton "Face" Peck
3. Captain H.M. Murdock
4. Sergeant Bosco "B.A." Baracus

These characters have become iconic in popular culture, known for their distinctive personalities and teamwork. 

Final Answer: The members of the A-Team are Colonel John "Hannibal" Smith, Lieutenant Templeton "Face" Peck, Captain H.M. Murdock, and Sergeant Bosco "B.A." Baracus.

> Finished chain.


{'input': 'Who are the members of the A-Team?',
 'output': 'The members of the A-Team are Colonel John "Hannibal" Smith, Lieutenant Templeton "Face" Peck, Captain H.M. Murdock, and Sergeant Bosco "B.A." Baracus.'}

In [11]:
agent.invoke(input="When does the restaurant open?")



> Entering new AgentExecutor chain...
I need to find out the opening hours of the restaurant.  
Action: restaurant search  
Action Input: "opening hours"  
Observation: Q: What are the hours of operation for Bella Vista?
Sundays, we welcome guests from 12 p.m. to 10 p.m.
during weekends and holidays, to ensure a seamless dining experience.
A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends
Thought:I see that Sundays start at 12 p.m. but need more information about the other days of the week to provide a complete answer.  
Action: restaurant search  
Action Input: "opening hours Monday to Saturday"  
Observation: Sundays, we welcome guests from 12 p.m. to 10 p.m.
during weekends and holidays, to ensure a seamless dining experience.
A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends
Q: What are the hours of operation for Bella Vista?
Thought:It seems the previous search didn't provide the 

{'input': 'When does the restaurant open?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### Chat Agents

In [12]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model="gpt-4o-mini")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

/var/folders/d1/d31w101x0f31vstymvwz8kwr0000gn/T/ipykernel_40524/2151239754.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [13]:
print(agent_chain.agent.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations a

In [14]:
agent_chain.invoke(input="What is the capital of France?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of France is Paris."
}
```

> Finished chain.


{'input': 'What is the capital of France?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})],
 'output': 'The capital of France is Paris.'}

In [15]:
agent_chain.invoke(input="Any suggestions what to visit there?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Some popular places to visit in Paris include the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. You might also want to take a stroll along the Seine River and visit the Palace of Versailles if you have time!"
}
```

> Finished chain.


{'input': 'Any suggestions what to visit there?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Any suggestions what to visit there?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Some popular places to visit in Paris include the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. You might also want to take a stroll along the Seine River and visit the Palace of Versailles if you have time!', additional_kwargs={}, response_metadata={})],
 'output': 'Some popular places to visit in Paris include the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. You might also want to take a stroll along the Seine River and visit the Palace of Versailles if you have time!'}